In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# DE labels
kaggle_train_de_df = pd.read_parquet('data/de_train.parquet')

In [3]:
genes = list(set(kaggle_train_de_df.columns.tolist()) - set(["cell_type", "sm_name", "sm_lincs_id", "SMILES", "control"]))

In [4]:
adata_test_df = pd.read_parquet('data/df_expr_predict.all_donors.parquet')

In [5]:
meta_data_columns = ['donor_id', 'cell_type', 'sm_name']
adata_obs_meta_df = pd.read_csv('data/df_meta_predict.all_donors.tsv', delimiter='\t')[meta_data_columns]

In [6]:
adata_test_df['donor_id'] = adata_obs_meta_df['donor_id'].tolist()
adata_test_df['cell_type'] = adata_obs_meta_df['cell_type'].tolist()
adata_test_df['sm_name'] = adata_obs_meta_df['sm_name'].tolist()

In [7]:
adata_test_df = adata_test_df.groupby(["donor_id", "cell_type", "sm_name"])[genes].sum().reset_index()

In [9]:
adata_test_df.reset_index(inplace=True)

In [10]:
adata_test_df[genes] = adata_test_df[genes].apply(lambda x: np.log(x / x.sum() + 1), axis=1)

In [11]:
# adata_test_df.to_parquet('data/adata_test_df_301123.parquet', engine='pyarrow')

In [23]:
adata_test_df = pd.read_parquet('data/adata_test_df_301123.parquet', engine='pyarrow')

In [24]:
neg_df = pd.read_parquet('data/neg.parquet', engine='pyarrow')

In [25]:
new_gene_names = [g + "_neg" for g in genes]
merged_df = adata_test_df.merge(neg_df, on=["donor_id", "cell_type"], suffixes=('', '_neg'))

In [26]:
adata_test_df[genes] = pd.DataFrame(adata_test_df[genes].to_numpy() - merged_df[new_gene_names].to_numpy())

In [27]:
ddd = adata_test_df.groupby(["cell_type", "sm_name"])[genes].mean().reset_index()

In [30]:
ddd.to_parquet('data/ddd_test.parquet', engine='pyarrow')

In [31]:
ddd.shape

(255, 18213)

In [32]:
len(genes)

18211